<a href="https://colab.research.google.com/github/endorgobio/2024_returnability/blob/main/20241011_Modelo_concreto_Gurobi_Pablo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color='056938'> **Cargar librerias y paquetes** </font>

In [1]:
!pip install gurobipy
!pip install gurobipy_pandas
!pip install requests
import requests, json
import numpy as np
import pandas as pd
import re
import math
import gurobipy as gp
from gurobipy import GRB
import gurobipy_pandas as gpd
import os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 37.1 MB/s eta 0:00:00


Carga el modulo con todas las funciones que creamos para generar la instancia

In [2]:
# url of the Python module utilities
url = 'https://raw.githubusercontent.com/endorgobio/2024_returnability/refs/heads/main/utilities.py'
response = requests.get(url) #Download the module from the URL

# Save the module locally in Colab
with open('utils.py', 'wb') as f:
    f.write(response.content)

import utils # import the module


# <font color='056938'> **Descripción del problema** </font>

# <font color='056938'> **Modelación** </font>

# <font color='056938'> **Implementación** </font>


Este notebook implementa en `gurobipy` este [modelo](https://drive.google.com/file/d/157MOs8L0B9B8zzu7aKABhcqFv1hq9Chp/view?usp=sharing) que estaba implementado en `pyomo`

La implementación usa las funciones descritas en [este notebook](https://drive.google.com/file/d/1ZGDer4HogMadqSBqyzykE8JiQ1upX50S/view?usp=sharing) y que se cargaron a través de el modulo `utils`



## <font color='8EC044'> **Generar  instancia** </font>

Se definen los parámetros y se usa la función cargada desde el modulo utils para generar la instancia




### <font color='46B8A9'> **Parámetros de entrada** </font>

Se leen los archivos de las localizaciones y las distancias y despues se crea un diccionario  de  parámetros (parameters) con todos los valores de entrada


#### <font color='260656'> **Leer archivos base** </font>

Leer archivos base de localizaciones y distancias

In [3]:
df_coord = pd.read_csv('https://docs.google.com/uc?export=download&id=14K0eAyrJlAkjvJ9OZMk7qy_dS0RyUSgl') # coordenadas
df_dist = pd.read_csv('https://docs.google.com/uc?export=download&id=15WHvGj0gg42q-rJPURlCBSOlSMoNGuzL') # distancias
df_demand = pd.read_csv('https://docs.google.com/uc?export=download&id=1w0PMK36H4Aq39SAaJ8eXRU2vzHMjlWGe') # demandas escenario base


#### <font color='260656'> **Definir parámetros** </font>

In [4]:
# Define the parameters in a flat dictionary
parameters = {
    # Basic parameters
    "n_acopios": 10,               # maximum 344
    "n_centros": 5,                # maximum 5
    "n_plantas": 3,                # maximum 3
    "n_productores": 5,            # maximum 5
    "n_envases": 3,
    "n_periodos": 5,

    # Technical parameters
    "ccv": 337610,  #130*2597                  # Classification capacity of the valorization centers
    "acv": 418117, # 161*2597                    # Storage capacity of the valorization centers
    "lpl": 168805, # 65*2597                     # Washing capacity of the washing plants
    "apl": 623280, #240*2597                    # Storage capacity of the washing plants
    "ta": 1, # 0.95,                    # Approval rate in valorization centers
    "tl": 1, # 0.90,                     # Approval rate in washing plants

    # Cost parameters
    "arr_cv": 5100000,            # Rental cost of valorization centers
    "arr_pl": 7000000,            # Rental cost of washing plants
    "renta_increm": 0.0069,        # Annual rent increase
    "ade_cv": 20000000,           # Adaptation cost of valorization centers
    "ade_pl": 45000000,           # Adaptation cost of washing plants
    "adecua_increm": 0.0069,       # Annual adaptation cost increase
    "qc": 140, # 363580/2597,                  # Classification and inspection cost
    "qt": 0.81, # 2120/2597,                    # Crushing cost
    "ql": 210, # 545370/2597,                  # Washing cost
    "qb": 140, # 363580/2597,                  # Laboratory test cost
    "qa": 140, # 363580/2597,                  # Transportation cost
    "cinv": 12.20, # 31678/2597,                 # Inventory cost of valorization centers
    "pinv": 11.20, # 29167/2597,                 # Inventory cost of washing plants

    # Environmental parameters
    "em": 0.0008736,               # CO2 emissions in kilometers
    "el": 0.002597,                # CO2 emissions in the washing process
    "et": 0.001096,                # CO2 emissions in the crushing process
    "en": 820.65,                 # CO2 emissions in the production of new containers

    # Contextual parameters
    "wa": 0.01,                    # WACC
    "recup_increm": 0, # 0.0025,        # Recovery rate increase
    "enr": 1039.66, # 2700000,                # Price of returnable container
    "tri": 200, # 300000,                 # Price of crushed container
    "adem": 0.01,                  # Demand increase
    "recup": 1, # 0.89,                 # Recovery rate
    "envn": 1250, # 3246250,               # Price of new containers
    "dep": 70, # 181790,                 # Deposit cost
    "n_pack_prod": 2,              # maximum number of containers that use each producer
    "dem_interval": [1000, 2000],     # interval in which the demand lies

    # Dataframes
    'df_coord': df_coord, # coordenadas
    'df_dist': df_dist, # distancias
    'df_demand': df_demand, # demandas escenario base

    # Optional = None
    'type_distance' : 'distance_geo',
    'initial_demand': None,
}

### <font color='46B8A9'> **invocar función `create_instance()` del modulo `utils`** </font>

invocamos la función `create_instance()` que hace parte del modulo `utils` que cargamos al inicio del notebook

In [6]:
instance = utils.create_instance(parameters)

Para correr el caso base con las demandas calculadas y no las generadas aleatoriamente, debes agregar el parametro `initial_demand` al llamado de la función, así:


1. Lees las demandas del esecanrio base

In [ ]:
initial_demand =  utils.read_dem_initial(df_demand)
initial_demand

2. Modificas los parámetros que se requiera en los datos del directorio `parameters`.


```python
parameters[n_acopios] = 344
parameters[n_centros] = 5
parameters[n_plantas] = 3
parameters[n_producer] = 5
parameters[n_envases] = 3
parameters[initial_demand] = initial_demand

```



3. Creas nuevamente la instancia



```python
instance = utils.create_instance(parameters)
```



## <font color='8EC044'> **Crear modelo** </font>

In [7]:
# Función que crea el modelo

def create_model(instance,
                 model_integer = False # when True the model considers integer variables (False by default)
                 ):

  # read instance data
  n_acopios = instance['n_acopios']
  n_centros = instance['n_centros']
  n_plantas = instance['n_plantas']
  n_productores = instance['n_productores']
  n_envases = instance['n_envases']
  n_periodos = instance['n_periodos']
  renta_increm = instance['renta_increm']
  adecua_increm = instance['adecua_increm']
  qc = instance['qc']
  qt = instance['qt']
  ql = instance['ql']
  qb = instance['qb']
  ta = instance['ta']
  tl = instance['tl']
  qa = instance['qa']
  wa = instance['wa']
  em = instance['em']
  el = instance['el']
  et = instance['et']
  en = instance['en']
  recup_increm = instance['recup_increm']
  ccv = instance['ccv']
  acv = instance['acv']
  lpl = instance['lpl']
  apl = instance['apl']
  # dimdist = instance['dimdist']
  arr_cv = instance['arr_cv']
  arr_pl = instance['arr_pl']
  ade_cv = instance['ade_cv']
  ade_pl = instance['ade_pl']
  dep = instance['dep']
  enr = instance['enr']
  tri = instance['tri']
  adem = instance['adem']
  # demanda = instance['demanda']
  recup = instance['recup']
  cinv = instance['cinv']
  pinv = instance['pinv']
  envn = instance['envn']


  # Assign sets and other generated variables from the dictionary
  acopios = instance['acopios']
  centros = instance['centros']
  plantas = instance['plantas']
  productores = instance['productores']
  envases = instance['envases']
  periodos = instance['periodos']

  # Assign parameters and calculations from the dictionary
  cc = instance['cc']
  ca = instance['ca']
  cp = instance['cp']
  cl = instance['cl']
  # coord_acopios = instance['coord_acopios']
  # coord_centros = instance['coord_centros']
  # coord_plantas = instance['coord_plantas']
  # coord_productores = instance['coord_productores']
  da = instance['da']
  dl = instance['dl']
  dp = instance['dp']
  rv = instance['rv']
  rl = instance['rl']
  av = instance['av']
  al = instance['al']
  qd = instance['qd']
  pv = instance['pv']
  pt = instance['pt']
  b = instance['b']
  # env_pdtor = instance['env_pdtor']
  dem = instance['dem']
  de = instance['de']
  gi = instance['gi']
  ge = instance['ge']
  iv = instance['iv']
  il = instance['il']
  ci = instance['ci']
  cv = instance['cv']
  pe = instance['pe']
  a = instance['a']


  model = gp.Model('CircularEconomy')


  # Define variables
  x = model.addVars(centros, periodos, vtype=GRB.BINARY, name="x")
  y = model.addVars(centros, periodos, vtype=GRB.BINARY, name="y")
  z = model.addVars(plantas, periodos, vtype=GRB.BINARY, name="z")
  w = model.addVars(plantas, periodos, vtype=GRB.BINARY, name="w")


  if model_integer:
    q = model.addVars(envases, acopios, centros, periodos, vtype=GRB.INTEGER, name="q")
    r = model.addVars(envases, centros, plantas, periodos, vtype=GRB.INTEGER, name="r")
    combinations_u = [(p,k,l,t) for p,l,t in de.keys() for k in plantas]
    u = model.addVars(combinations_u, vtype=GRB.INTEGER, name="u")
    ic = model.addVars(envases, centros, periodos, vtype=GRB.INTEGER, name="ic")
    ip = model.addVars(envases, plantas, periodos, vtype=GRB.INTEGER, name="ip")
    combinations_er = [(p,l,t) for p,l,t in de.keys()]
    er = model.addVars(combinations_er, vtype=GRB.INTEGER, name="er")
  else:
    q = model.addVars(envases, acopios, centros, periodos, vtype=GRB.CONTINUOUS, name="q")
    r = model.addVars(envases, centros, plantas, periodos, vtype=GRB.CONTINUOUS, name="r")
    combinations_u = [(p,k,l,t) for p,l,t in de.keys() for k in plantas]
    u = model.addVars(combinations_u, vtype=GRB.CONTINUOUS, name="u")
    ic = model.addVars(envases, centros, periodos, vtype=GRB.CONTINUOUS, name="ic")
    ip = model.addVars(envases, plantas, periodos, vtype=GRB.CONTINUOUS, name="ip")
    combinations_er = [(p,l,t) for p,l,t in de.keys()]
    # er = model.addVars(combinations_er, vtype=GRB.CONTINUOUS, name="er")


  ## FUNCIÓN OBJETIVO
  # Componentes función objetivo
  model._ingreso_retornable = sum(u[p,k,l,t] * pv[p] for p in envases for k in plantas for l in productores for t in periodos if (p,k,l,t) in combinations_u)

  model._ingreso_triturado = (sum(r[p,j,k,t] * (1 - ta) / ta * pt[p] for p in envases for j in centros for k in plantas for t in periodos) +\
                              sum(u[p,k,l,t] * (1 - tl) / tl * pt[p] for p in envases for k in plantas for l in productores for t in
                              periodos if (p,k,l,t) in combinations_u))

  # model._egreso_envnuevo = sum(er[p,l,t] * pe[p] for p in envases for l in productores for t in periodos if (p,l,t) in combinations_er)

  model._egreso_adecuar = sum(y[j,t]*av[j,t] for j in centros for t in periodos) + sum(w[k,t]*al[k,t] for k in plantas for t in periodos)

  model._egreso_uso = sum(x[j,t]*rv[j,t] for j in centros for t in periodos) + sum(z[k,t]*rl[k,t] for k in plantas for t in periodos)

  model._egreso_transporte = sum(q[p,i,j,t]*qa*da[i,j] for p in envases for i in acopios for j in centros for t in periodos) +\
                      sum(r[p,j,k,t]*qa*dl[j,k] for p in envases for j in centros for k in plantas for t in periodos) +\
                      sum(u[p,k,l,t]*qa*dp[k,l] for p in envases for k in plantas for l in productores for t in periodos if (p,k,l,t) in combinations_u)

  model._egreso_compra = sum(q[p,i,j,t]*qd[p] for p in envases for i in acopios for j in centros for t in periodos) #depósito

  model._egreso_inspeccion = sum((r[p,j,k,t]/ta)*qc for p in envases for j in centros for k in plantas for t in periodos) +\
                              sum((u[p,k,l,t]/tl)*qc for p in envases for k in plantas for l in productores for t in periodos if (p,k,l,t) in combinations_u)

  model._egreso_lavado = sum((u[p,k,l,t]/tl)*ql for p in envases for k in plantas for l in productores for t in periodos if (p,k,l,t) in combinations_u)

  model._egreso_pruebas = sum(u[p,k,l,t]*qb for p in envases for k in plantas for l in productores for t in periodos if (p,k,l,t) in combinations_u)

  model._egreso_trituracion = (sum(r[p,j,k,t] * (1 - ta)/ ta * qt for p in envases for j in centros for k in plantas for t in periodos) +\
                               sum(u[p,k,l,t] * (1 - tl)/ tl * qt for p in envases for k in plantas for l in productores
                               for t in periodos if (p,k,l,t) in combinations_u))

  model._egreso_invcentros= sum(ic[p,j,t]*ci[j] for p in envases for j in centros for t in periodos)

  model._egreso_invplantas = sum(ip[p,k,t]*cv[k] for p in envases for k in plantas for t in periodos)

  model._emisiones_transporte = (sum(da[i,j]*q[p,i,j,t] for p in envases for i in acopios for j in centros for t in periodos) + \
                          sum(dl[j,k]*r[p,j,k,t] for p in envases for j in centros for k in plantas for t in periodos) + \
                          sum(dp[k,l]*u[p,k,l,t] for p in envases for k in plantas for l in productores for t in periodos if (p,k,l,t) in combinations_u))*em

  model._emisiones_lavado = sum((u[p,k,l,t]/tl)*el for p in envases for k in plantas for l in productores for t in periodos if (p,k,l,t) in combinations_u)

  model._emisiones_trituracion = (sum(r[p,j,k,t] * (1 - ta) / ta * et for p in envases for j in centros for k in plantas for t in periodos) + \
                                  sum(u[p,k,l,t] * (1 - tl) / tl * et for p in envases for k in plantas for l in productores
                                  for t in periodos if (p,k,l,t) in combinations_u))

  # model._emisiones_envnuevo = (sum(er[p,l,t]*en for p in envases for l in productores for t in periodos if (p,l,t) in combinations_er))

  # Agregar objetivo
  # funcion_objetivo = model._ingreso_retornable + model._ingreso_triturado - model._egreso_adecuar  - model._egreso_uso - model._egreso_envnuevo -\
  #             model._egreso_transporte - model._egreso_compra - model._egreso_inspeccion - model._egreso_lavado - model._egreso_pruebas -\
  #             model._egreso_trituracion - model._egreso_invcentros - model._egreso_invplantas

  funcion_objetivo = model._ingreso_retornable + model._ingreso_triturado - model._egreso_adecuar  - model._egreso_uso  -\
              model._egreso_transporte - model._egreso_compra - model._egreso_inspeccion - model._egreso_lavado - model._egreso_pruebas -\
              model._egreso_trituracion - model._egreso_invcentros - model._egreso_invplantas

  model.setObjective(funcion_objetivo,GRB.MAXIMIZE)

  # Restriccion 1: Capacidad de procesamiento centro de clasificación
  model.addConstrs((gp.quicksum(r[p,j,k,t] / ta for p in envases for k in plantas) <= cc[j] * x[j,t] for j in centros for t in periodos),
                  name='cap_proc_centros')

  # Restriccion 2: Capacidad de procesamiento plantas de lavado
  model.addConstrs((gp.quicksum(u[p,k,l,t] / tl for p in envases for l in productores if  (p,k,l,t) in combinations_u) <= cl[k]*z[k,t]
                   for k in plantas for t in periodos),name='cap_proc_plantas')

  # Restriccion 3: Cumplimiento de demanda
  # model.addConstrs((gp.quicksum(u[p,k,l,t] for k in plantas) + er[p,l,t] == de[p,l,t] for p in envases for l in productores
  #                  for t in periodos if (p,l,t) in er),name='demanda')
  model.addConstrs((gp.quicksum(u[p,k,l,t] for k in plantas) <= de[p,l,t] for p in envases for l in productores
                   for t in periodos if (p,l,t) in de),name='demanda')

  # Restriccion 4: No debe recogerse más de la generación
  model.addConstrs((gp.quicksum(q[p,i,j,t] for j in centros) <= ge[p,i,t] for p in envases for i in acopios for t in periodos),
                  name='no_recoger_mas_gen')

  ## Adecuación y apertura centros de clasificacion
  # Restriccion 5:
  model.addConstrs((x[j,t] >= y[j,tp] for j in centros for t in periodos for tp in periodos if t >=tp),
                  name='mantener_abierto_centro')

  # Restriccion 6:
  model.addConstrs((gp.quicksum(y[j,tp] for tp in range(1,t+1)) >= x[j,t] for j in centros for t in periodos),
                  name='usar_cuando_centro')

  # Restriccion 7
  model.addConstrs((gp.quicksum(y[j,t] for t in periodos) <= 1 for j in centros),
                  name='adecuar_centro')

  ## Adecuación y apertura plantas de lavado
  # Restriccion 8:
  model.addConstrs((z[k,t] >= w[k,tp] for k in plantas for t in periodos for tp in periodos if t >=tp),
                  name='mantener_abierta_planta')

  # Restriccion 9
  model.addConstrs((gp.quicksum(w[k,tp] for tp in range(1,t+1)) >= z[k,t] for k in plantas for t in periodos),
                  name='usar_cuando_planta')

  # Restriccion 10
  model.addConstrs((gp.quicksum(w[k,t] for t in periodos) <= 1 for k in plantas),
                  name='adecuar_planta')

  # Restriccion 11: Inventario en centros de clasificación
  model.addConstrs(
      ((ic[p,j,t]==ic[p,j,t-1] + gp.quicksum(q[p,i,j,t] for i in acopios)-gp.quicksum(r[p,j,k,t]/ta for k in plantas)) if t >1
      else (ic[p,j,t]==iv[p,j] + gp.quicksum(q[p,i,j,t] for i in acopios)-gp.quicksum(r[p,j,k,t]/ta for k in plantas))
      for p in envases for j in centros for t in periodos ),
      name='inv_centros')

  # Restricción 12: Capacidad de almacenamiento en centros de clasificación
  model.addConstrs((gp.quicksum(ic[p,j,t] for p in envases) <= ca[j]*x[j,t] for j in centros for t in periodos),
                  name='cap_alm_centros')

  # Restriccion 13: Inventario en plantas de lavado
  model.addConstrs(
      ((ip[p,k,t]==ip[p,k,t-1] + gp.quicksum(r[p,j,k,t] for j in centros)-gp.quicksum(u[p,k,l,t]/tl for l in productores if (p,k,l,t) in combinations_u)) if t >1
      else (ip[p,k,t]==il[p,k] + gp.quicksum(r[p,j,k,t] for j in centros)-gp.quicksum(u[p,k,l,t]/tl for l in productores if (p,k,l,t) in combinations_u))
      for p in envases for k in plantas for t in periodos ),
      name='inv_plantas')

  # Restricción 14: Capacidad de almacenamiento en plantas de lavado
  model.addConstrs((gp.quicksum(ip[p,k,t]  for p in envases) <= cp[k]*z[k,t] for k in plantas for t in periodos),
                  name='cap_alm_centros')

  return model

## <font color='8EC044'> **Resolver y obtener solución** </font>

In [8]:
# Run optimization
model = create_model(instance)

# Optimize
model.optimize()
if model.status == GRB.OPTIMAL:
    print(f"Objective value: {model.ObjVal}")
else:
    print("Optimization was not successful")


Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 568 rows, 1325 columns and 3481 nonzeros
Model fingerprint: 0x3e59684b
Variable types: 1245 continuous, 80 integer (80 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+05]
  Objective range  [1e+01, 5e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+03]
Found heuristic solution: objective -0.0000000

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 1: -0 
No other solutions better than -0

Optimal solution found (tolerance 1.00e-04)
Best objective -0.000000000000e+00, best bound -0.000000000000e+00, gap 0.0000%
Objective value: -0.0


### <font color='46B8A9'> **Obtener el valor de las variables** </font>

La función `get_vars_sol`, importada en `utils`, extrae las variables de un modelo de optimización, organizándolas por grupos y creando DataFrames con sus índices y valores. Los nombres de las columnas se definen según el tipo de variable, permitiendo una representación clara de los resultados.


In [ ]:
dict_df = utils.get_vars_sol(model)
df_u = dict_df['u']
df_u

### <font color='46B8A9'> **Obtener los componentes de la FO** </font>

La función `get_obj_components`, importada en `utils`, calcula los valores de componentes específicos de la función objetivo de un modelo, como ingresos, costos y emisiones. Retorna un diccionario que contiene tanto el valor total de la utilidad como los valores individuales de cada componente.

In [ ]:
utils.get_obj_components(model)

# <font color='056938'> **Validación 1** </font>

Consideremos el caso en el que:  

* La adecuación y uso de la infraestructura es gratis
* Todos los procesos tienen costo `0`
* No hay tasas de pérdida en los proceso



In [38]:
# Set parameters
parameters["ta"]= 1 # Approval rate in valorization centers
parameters["tl"]= 1 # Approval rate in washing plants
parameters['arr_cv']= 0  # Rental cost of valorization centers
parameters['arr_pl']= 0  # Rental cost of washing plants
parameters['ade_cv']= 0 # Adaptation cost of valorization centers
parameters['ade_pl']= 0 # Adaptation cost of washing plants
parameters['qc']= 0 # Classification and inspection cost
parameters['qt']= 0 # Crushing cost
parameters['ql']= 0 # Washing cost
parameters['qb']= 0 # Laboratory test cost
parameters['qa']= 0 # Transportation cost
parameters['cinv']= 0 # Inventory cost of valorization centers
parameters['pinv']= 0 # Inventory cost of washing plants
parameters['recup'] = 0.9 # Recovery rate
parameters['recup_increm'] = 0 # Recovery rate increase

instance = utils.create_instance(parameters, seed=42)

# get aggregated demand
df_demands = [[k[0], k[1], k[2], value] for k, value in instance['de'].items()]
df_demands = pd.DataFrame(df_demands, columns=['envase', 'productor', 'periodo', 'demanda'])
df_gen = [[k[0], k[1], k[2], value] for k, value in instance['de'].items()]
df_gen = pd.DataFrame(df_gen, columns=['envase', 'productor', 'periodo', 'generacion'])
df_demands['generacion'] = df_gen['generacion']
df_demands['periodo'] = df_demands['periodo'].astype(int)

grouped_df = df_demands.groupby(['envase', 'periodo']).agg(
    demanda=('demanda', 'sum'),   # Custom name 'Total_Value' for the sum of 'Value1'
    generacion=('generacion', 'sum') # Custom name 'Average_Value' for the mean of 'Value2'
).reset_index()


grouped_df


,envase,periodo,demanda,generacion
0,E0,1,5593,5593
1,E0,2,5646,5646
2,E0,3,5704,5704
3,E0,4,5761,5761
4,E0,5,5819,5819
5,E1,1,4581,4581
6,E1,2,4625,4625
7,E1,3,4671,4671
8,E1,4,4718,4718
9,E1,5,4766,4766


check that capcities are larger than demands

In [39]:
# get cummulative
grouped = df_demands.groupby(['periodo'])['demanda'].sum().reset_index()
grouped['storage_wash'] = instance['apl']
grouped['class_cap'] = instance['ccv']
grouped['storage_class'] = instance['acv']
grouped['washing_cap'] = instance['lpl']
grouped['storage_wash'] = instance['apl']
grouped

,periodo,demanda,storage_wash,class_cap,storage_class,washing_cap
0,1,15191,623280,337610,418117,168805
1,2,15337,623280,337610,418117,168805
2,3,15491,623280,337610,418117,168805
3,4,15646,623280,337610,418117,168805
4,5,15805,623280,337610,418117,168805


Ahora corramoslo para distintas combinaciones de tasa de recuperación y el incremento de esta

In [40]:
for recu, delta in  [(1, 0), (0.9, 0), (0.9, 0.02)]:
  parameters['recup'] = recu
  parameters['recup_increm'] = delta
  # create instance
  instance = utils.create_instance(parameters, seed=42)

  # Run optimization
  model = create_model(instance)
  model.optimize()

  # get solution
  dict_df = utils.get_vars_sol(model)
  df_u = dict_df['u']
  grouped = df_u.groupby(['envase', 'periodo'])['cantidad'].sum().reset_index()
  grouped_df[str(recu)+"-"+str(delta)] = grouped['cantidad']
grouped_df

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 568 rows, 1325 columns and 3481 nonzeros
Model fingerprint: 0x6c28aa16
Variable types: 1245 continuous, 80 integer (80 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+05]
  Objective range  [7e+01, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective -0.0000000
Presolve removed 560 rows and 1306 columns
Presolve time: 0.03s
Presolved: 8 rows, 19 columns, 30 nonzeros
Found heuristic solution: objective 5.477609e+07
Variable types: 19 continuous, 0 integer (0 binary)

Root relaxation: objective 7.452419e+07, 6 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incu

,envase,periodo,demanda,generacion,1-0,0.9-0,0.9-0.02
0,E0,1,5593,5593,5593.0,5033.7,5033.700000
1,E0,2,5646,5646,5593.0,5033.7,5134.374000
2,E0,3,5704,5704,5531.0,3819.9,4766.808455
3,E0,4,5761,5761,5761.0,5761.0,5761.000000
4,E0,5,5819,5819,5819.0,5819.0,5819.000000
5,E1,1,4581,4581,4581.0,4122.9,4122.900000
6,E1,2,4625,4625,4581.0,4122.9,4205.358000
7,E1,3,4671,4671,4530.0,3128.6,3904.100749
8,E1,4,4718,4718,4718.0,4718.0,4718.000000
9,E1,5,4766,4766,4766.0,4766.0,4766.000000


In [23]:
# get aggregated demand
df_demands = [[k[0], k[1], k[2], value] for k, value in instance['de'].items()]
df_demands = pd.DataFrame(df_demands, columns=['envase', 'productor', 'periodo', 'demanda'])
df_demands['periodo'] = df_demands['periodo'].astype(int)

# get cummulative
grouped = df_demands.groupby(['periodo'])['demanda'].sum().reset_index()
grouped['class_cap'] = instance['ccv']
grouped['storage_class'] = instance['acv']
grouped['washing_cap'] = instance['lpl']
grouped['storage_wash'] = instance['apl']
grouped



,periodo,demanda,class_cap,storage_class,washing_cap,storage_wash
0,1,15191,337610,418117,168805,623280
1,2,15337,337610,418117,168805,623280
2,3,15491,337610,418117,168805,623280
3,4,15646,337610,418117,168805,623280
4,5,15805,337610,418117,168805,623280


In [26]:
grouped = df_demands.groupby(['envase', 'periodo'])['demanda'].sum().reset_index()
grouped

,envase,periodo,demanda
0,E0,1,5593
1,E0,2,5646
2,E0,3,5704
3,E0,4,5761
4,E0,5,5819
5,E1,1,4581
6,E1,2,4625
7,E1,3,4671
8,E1,4,4718
9,E1,5,4766


In [24]:
df_E0 = df[df['envase']=='E0']
df_E0

,envase,periodo,demanda,1-0,0.9-0,0.9-0.02
0,E0,1,5593,5593.0,5033.7,5033.700000
1,E0,2,5646,5593.0,5033.7,5134.374000
2,E0,3,5704,5531.0,3819.9,4766.808455
3,E0,4,5761,5761.0,5761.0,5761.000000
4,E0,5,5819,5819.0,5819.0,5819.000000


In [18]:
import plotly.graph_objects as go



# Create traces for each line
trace1 = go.Scatter(x=df_E0['periodo'], y=df_E0['demanda'], mode='lines', name='Line 1')
trace2 = go.Scatter(x=df_E0['periodo'], y=df_E0['1-0'], mode='lines', name='Line 2')
trace3 = go.Scatter(x=df_E0['periodo'], y=df_E0['0.9-0'], mode='lines', name='Line 2')
trace4 = go.Scatter(x=df_E0['periodo'], y=df_E0['0.9-0.02'], mode='lines', name='Line 2', line=dict(color='red'))

# Create the figure with the traces
fig = go.Figure(data=[trace1, trace2, trace3])

# Customize layout
fig.update_layout(
    title='Multi-Line Chart with Plotly',
    xaxis_title='X Axis',
    yaxis_title='Y Axis',
    legend_title='Lines',
    template='plotly_white'
)

# Show the plot
fig.show()


In [ ]:
df_u = dict_df['u']
grouped = df_u.groupby(['envase', 'periodo'])['cantidad'].sum().reset_index()
grouped

In [ ]:
df_u = dict_df['u']
grouped = df_u.groupby(['productor', 'envase', 'periodo'])['cantidad'].sum().reset_index()
grouped['periodo'] = grouped['periodo'].astype(int)
grouped[(grouped['productor']=='P0') & (grouped['envase']=='E0')].sort_values(by=['envase','periodo'])



In [10]:
# get aggregated demand
agg_dem = {(env, t):0 for env in instance['envases'] for t in instance['periodos']}
for key, value in instance['de'].items():
  agg_dem[(key[0], key[2])] += value

df = pd.DataFrame([[key[0], key[1], value] for key,value in agg_dem.items()], columns=['envase', 'periodo', 'demanda'])
df['periodo'] = df['periodo'].astype(int)
df

,envase,periodo,demanda
0,E0,1,5593
1,E0,2,5646
2,E0,3,5704
3,E0,4,5761
4,E0,5,5819
5,E1,1,4581
6,E1,2,4625
7,E1,3,4671
8,E1,4,4718
9,E1,5,4766


Por ejemplo podemos consultar todos los valores de las variables `x`, así:

In [ ]:
!pip install openpyxl

import pandas as pd

# Configuraciones opcionales para ver todo el contenido del DataFrame
pd.set_option('display.max_rows', None)  # Sin límite de filas
pd.set_option('display.max_columns', None)  # Sin límite de columnas

# Asumiendo que dict_df['r'] ya contiene tu DataFrame
df_r = dict_df['r']

# Exportar a Excel
df_r.to_excel('df_r.xlsx', index=False)  # 'index=False' para no exportar el índice



### <font color='46B8A9'> **Obtener el de las componentes de la función objetivo** </font>

In [ ]:
# Get the name for the OF components
components = [
    '_ingreso_retornable',
    '_ingreso_triturado',
    '_egreso_envnuevo',
    '_egreso_adecuar',
    '_egreso_uso',
    '_egreso_transporte',
    '_egreso_compra',
    '_egreso_inspeccion',
    '_egreso_lavado',
    '_egreso_pruebas',
    '_egreso_trituracion',
    '_egreso_invcentros',
    '_egreso_invplantas',
    '_emisiones_transporte',
    '_emisiones_lavado',
    '_emisiones_trituracion',
    '_emisiones_envnuevo'
]

data_FO = {}

data_FO["utilidad_total"]= model.ObjVal

for attr in components:
  expr = getattr(model, attr)
  # Ensure the attribute is an expression
  if isinstance(expr, gp.LinExpr):
    value = expr.getValue()
    data_FO[attr]= expr.getValue() # add expr if you want to get the expression
  else:
    data_FO[attr] = value

data_FO

Por ejemplo el costo de adecuar es:

In [ ]:
data_FO['_egreso_adecuar']

## <font color='8EC044'> **Múltiples corridas** </font>

Consideremos el caso en el que deseamos correr el modelo con diferentes números de acopios (`n_acopios`). Asumiendo que partimos del conjunto inicial de parámetros que ya habiamos definido, debemos:

* Crear un ciclo que vaya cambiando el valor de el parámetro `n_acopios` en el diccionario `parameters`
* En cada iteración del ciclo una vez cambiamos el parámetro, debemos volver a crear la instancia con la funcipon `create_instance()`
* En cada iteración resolvemos el modelo y guardamos los resultados que deseemos





In [ ]:
experiments = [] # to save the results

for n_acopios_val in [5, 10, 15]:

  # Create the instance
  parameters['n_acopios'] = n_acopios_val # Change the value of n_acopios
  instance = utils.create_instance(parameters) # create the instance again

  # Run optimization
  model = create_model(instance)
  model.optimize()

  # Get results
  if model.status == GRB.OPTIMAL:
    experiments.append([n_acopios_val, model.ObjVal])
  else:
    print("Optimization was not successful")


Asumiendo que solo guaradamos el valor del parámetro n_acopios y el rescpectivo valor de la función objetivo. Tendríamos



In [ ]:
df_resultados = pd.DataFrame(experiments, columns=['n_acopios', 'utilidad'])
df_resultados

Nota que igual podrías requerir guardar mucha más información sobre cada corrida. Para eso usuarias las funciones que ya creamos anteriromente para extraer la solución